In [ ]:
import socket
scheduler_ip = socket.gethostbyname(socket.gethostname())
scheduler_port = 30000
scheduler_address = str(scheduler_ip) + ":" + str(scheduler_port)
scheduler_address

In [8]:
from dask_kubernetes import KubeCluster

In [ ]:
# %load config/worker.yaml
kind: Pod
metadata:
  labels:
    role: worker
spec:
  restartPolicy: Never
  containers:
  - image: daskdev/dask:latest
    imagePullPolicy: Always
    args: [dask-worker, $(DASK_SCHEDULER_ADDRESS), --nthreads, '1', --no-bokeh, --memory-limit, 2GB, --death-timeout, '60']
    name: dask-worker
    resources:
      requests:
        cpu: "1"
        memory: 2G
      limits:
        cpu: "1"
        memory: 2G


In [ ]:
cluster = KubeCluster.from_yaml('config/worker.yaml',
                                port=scheduler_port,
                                env={'DASK_SCHEDULER_ADDRESS': scheduler_address})

In [ ]:
cluster.scheduler_address

In [ ]:
cluster.scale(2)

In [ ]:
from dask.distributed import Client
client = Client(cluster)

In [ ]:
from operator import add
answer = client.get({'x': (add, 1, 2)}, 'x')
print("the answer is:", answer)